In [73]:
import json
import random
import pathlib
from datasets import load_dataset

In [74]:
test_templates = list()
for file_name in pathlib.Path("./batch_results/").glob("*.jsonl"):
    print(file_name)
    with open(file_name, "r") as f:
        for line in f.readlines():
            instance = json.loads(line)
            custom_id = instance['custom_id']
            problem_id = custom_id.split('-')[1]
            template = instance['response']['body']['choices'][0]['message']['content']
            if template not in test_templates:
                test_templates.append(template)
                with open(f"cpp_templates/test_template_{problem_id}.cpp", "w") as f:
                    f.write(template)

batch_results/batch_6800f3c34b90819085ff1d097f977cea_output.jsonl


In [69]:
venus_ds = load_dataset("Elfsong/Venus", "cpp", split='train')
venus_dict = {}
for instance in venus_ds:
    venus_dict[int(instance['question_id'])] = instance

In [70]:
leetcode_ds = load_dataset("Elfsong/leetcode_data", split='train')
leetcode_dict = {}
for instance in leetcode_ds:
    leetcode_dict[int(instance['problem_id'])] = instance

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

In [71]:
for template_name in pathlib.Path("./cpp_templates/").glob("*.cpp"):
    if "template" not in template_name.stem: continue
    problem_id = int(template_name.stem.split("_")[2])

    if problem_id not in leetcode_dict: continue
    if problem_id not in venus_dict: continue

    leetcode_instance = leetcode_dict[problem_id]
    venus_instance = venus_dict[problem_id]

    solutions = venus_instance['rt_list'] + venus_instance['mm_list']
    if len(solutions) == 0: continue
    solution_code = random.choice(solutions)['code']

    test_cases = json.loads(leetcode_instance['test_cases'])
    test_case_str = json.dumps(test_cases, separators=(',', ':'))
    test_case_str_literal = f'R"({test_case_str})"'

    template_code = open(template_name, "r").read()
    template_code = template_code.replace("==Solution Code==", solution_code)
    template_code = template_code.replace("==Test Cases==", test_case_str_literal)

    with open(f"cpp_templates/test_{problem_id}.cpp", "w") as f:
        f.write(template_code)